### MNIST Handwriting recognition using CNN
Conv2d & Maxpool

helpfull website : 
Conv2d, ConvTranspose2d, MaxPool2d Calculator : http://layer-calc.com/ 

In [223]:
import torch
import torchvision.datasets as data
import torchvision.transforms as transforms
import random

In [208]:
##Hardware
import torch
if torch.cuda.is_available() == True:
  device = 'cuda'
  templist = [1,2,3]
  templist = torch.FloatTensor(templist).to(device)
  print("Cuda torch working : ",end="")
  print(templist.is_cuda)
  print("current device no. : ",end="")
  print(torch.cuda.current_device())
  print("GPU device count : ",end="")
  print(torch.cuda.device_count())
  print("GPU name : ",end="")
  print(torch.cuda.get_device_name(0))
  print("device : ",device)
  ! nvidia-smi
elif torch.backends.mps.is_available() == True:
  print("Apple device detected\nActivating Apple Silicon GPU")
  device = torch.device("mps")
else:
  print("cant use gpu , activating cpu")
  device = 'cpu'

Cuda torch working : True
current device no. : 0
GPU device count : 1
GPU name : NVIDIA GeForce GTX 1080
device :  cuda
Mon Jan 30 01:24:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 528.02       Driver Version: 528.02       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| 42%   29C    P8    12W / 180W |   7987MiB /  8192MiB |     17%      Default |
|                               |                      |                  N/A |
+-------------------------------+---------------

In [209]:
class CNN(torch.nn.Module):
  """
  def __init__(self):
  
    super(CNN,self).__init__()
    
    self.conv1 = torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1)
    self.relu = torch.nn.ReLU()
    self.maxpoo1 = torch.nn.MaxPool2d(kernel_size=2,stride=2)
    self.conv2 = torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1)
    self.relu2 = torch.nn.ReLU()
    self.maxpoo2 = torch.nn.MaxPool2d(kernel_size=2,stride=2)
    self.fc = torch.nn.Linear(7*7*64,10,bias=True) # this shows the input and output
    
  def forward(self,x):
    
    out = self.conv1(x)
    out = self.relu1(out)
    out = self.maxpoo1(out)
    out = self.conv2(out)
    out = self.relu2(out)
    out = self.maxpoo2(out)
    out = self.fc(out)
    
    return out
    """
"""    
method = input("1 : resnet // 2 : veg  // 3 L alexnet")
if method == '1':
  method = "resnet"
elif method == '2':
  method = "vgg"
elif method == '3':
  method = "alexnet"
else:
  print(" I didnt understand, just going to use alexnet anyway")
  method = "alexnet"
"""
"""
import torchvision.models as models

if method == "resnet":
    model = models.resnet18(pretrained=True)
    model.fc = torch.nn.Linear(in_features=512, out_features=10, bias=True)
elif method == "vgg":
    model = models.vgg16(pretrained=True)
    model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=10, bias=True)
elif method == "alexnet":
    model = models.alexnet(pretrained=True)
    model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=10, bias=True)
else:
    raise ValueError("Check Method!")
"""
class CNN(torch.nn.Module):
  def __init__(self,dropout_rate=0.2):
    super().__init__()
    self.dropout_rate = dropout_rate
    self.layer1 = torch.nn.Sequential(
      torch.nn.Conv2d(in_channels=1,out_channels=96,kernel_size=11,stride=4,padding=0),#227 to 55// (227-2*0-11)/4 + 1
      torch.nn.ReLU(),
      torch.nn.MaxPool2d(kernel_size=3,stride=2)#55 to 27
      )
    
    self.layer2 = torch.nn.Sequential(
      torch.nn.Conv2d(in_channels=96,out_channels=256,kernel_size=5,stride=1,padding=2),##27 to 27
      torch.nn.ReLU(),
      torch.nn.MaxPool2d(kernel_size=3,stride=2)#27 to 13
    ) # this shows the input and output
    self.layer3 = torch.nn.Sequential(
      torch.nn.Conv2d(256, 384, 3, 1, 1),#to 13
      torch.nn.ReLU()
    )
    self.layer4 = torch.nn.Sequential(
        torch.nn.Conv2d(384, 384, 3, 1, 1),#to 13
        torch.nn.ReLU()
    )
    self.layer5 = torch.nn.Sequential(
        torch.nn.Conv2d(384, 256, 3, 1, 1),#to 13
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(3, 2)#13 to 6
    )
    
    self.fc1 = torch.nn.Linear(256 * 6 * 6,4096)#256 nurons with 6*6 pic
    
    self.fc2 = torch.nn.Linear(4096,4096)
    self.fc3 = torch.nn.Linear(4096,10) #bias is true in default
    
  def forward(self,x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = self.layer5(out)
    out = out.view(out.size(0),-1)
    
    #print(f"out shape before fc1: {out.shape}")
    out = torch.nn.functional.relu(self.fc1(out))
    out = torch.nn.functional.dropout(out,self.dropout_rate)
    #print(f"out shape before fc2: {out.shape}")
    out = torch.nn.functional.relu(self.fc2(out))
    out = torch.nn.functional.dropout(out,self.dropout_rate)
    out = self.fc3(out)
    out = torch.nn.functional.log_softmax(out,dim=1)
    
    return out
model = CNN().to(device)

c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [210]:
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [211]:
LR_list = [10000,100,1,1e-2,1e-4,1e-6,1e-8]
learning_rate = [1e-3]
nb_epoch = 15
batch_size = 10
drop_out_posibility = 0.3
def show_parameters():
  print("Learning list : ",LR_list)
  print("Learning rate : ",learning_rate)
  print("Nb_epoch : ",nb_epoch)
  print("Batch size : ",batch_size)
  print("dropout : ",drop_out_posibility)
show_parameters()

Learning list :  [10000, 100, 1, 0.01, 0.0001, 1e-06, 1e-08]
Learning rate :  [0.001]
Nb_epoch :  15
Batch size :  10
dropout :  0.3


This code creates a "transformation pipeline" called "AlexTransform" using the transforms.Compose() function from the PyTorch library's transforms module. The pipeline is composed of three separate transformations that will be applied in sequence to any data passed through it.  

transforms.Resize((227, 227)): This transformation resizes the input data to have a height and width of 227 pixels each.  
transforms.ToTensor(): This transformation converts the input data from a PIL image to a PyTorch tensor.  
transforms.Normalize((0.1307,), (0.3081,)): This transformation normalizes the input data by subtracting the mean and dividing by the standard deviation. The mean and standard deviation are provided as arguments (0.1307 and 0.3081 respectively) which represent the mean and standard deviation of the MNIST dataset.  
So, in summary, the AlexTransform pipeline will resize any input data to (227, 227), convert it to a PyTorch tensor and normalize it using the mean and std of the MNIST dataset.  

In [212]:
AlexTransform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [220]:
# Creating and downloading the MNIST dataset

mnist_train= data.MNIST(root='MNIST_data',
                        train = True, #bring train data
                        transform=AlexTransform,
                        download=True)
mnist_test = data.MNIST(root='MNIST_data',
                        train=False,
                        transform=AlexTransform,
                        download=True)

In [222]:
mnist_train

Dataset MNIST
    Number of datapoints: 60000
    Root location: MNIST_data
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=(227, 227), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [214]:
LR_list = [10000,100,1,1e-2,1e-4,1e-6,1e-8]
learning_rate = [1e-3]
nb_epoch = 10
batch_size = 30
def show_parameters():
  print("Learning list : ",LR_list)
  print("Learning rate : ",learning_rate)
  print("Nb_epoch : ",nb_epoch)
  print("Batch size : ",batch_size)
show_parameters()

Learning list :  [10000, 100, 1, 0.01, 0.0001, 1e-06, 1e-08]
Learning rate :  [0.001]
Nb_epoch :  10
Batch size :  30


In [215]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size, #batch size at parameters
                                          shuffle=True, #mix it up
                                          drop_last=True) #len(data)%batch = left overs shoud I drop this or not?

In [216]:
#models = [['1layer','SGD'],['5layer','SGD'],['1layer','Adam'],['5layer','Adam'],['5layer+drop','SGD'],['5layer+drop','Adam']] 
models = ['Adam']
loss = torch.nn.CrossEntropyLoss().to(device)


def choose_model(LR,optim):
  global model
  global optimzier
  
  if optim == 'SGD':
    optimzier = torch.optim.SGD(model.parameters(),lr=LR)
    #print("SGD")
  elif optim == 'Adam':
    optimzier = torch.optim.Adam(model.parameters(),lr=LR)
    #print("Adam")
  else:
    print("Error unknown optim option please check models list")


choose_model(LR_list[0],models[0])

In [217]:
show_parameters()

Learning list :  [10000, 100, 1, 0.01, 0.0001, 1e-06, 1e-08]
Learning rate :  [0.001]
Nb_epoch :  10
Batch size :  30


In [218]:
from tqdm import tqdm
batch_len = len(data_loader)

for LR in LR_list:
  for optim in models:
    choose_model(LR,optim)#function chooseing model
    
    for epoch in tqdm(range(nb_epoch+1)):
      model.train()
      avg_cost = 0
      for X , Y in (data_loader):
        
        #change image to 1 dim list
        X = X.to(device)
        Y = Y.to(device)
        
        #reset grad
        optimzier.zero_grad()
        #foward
        hypothesis = model(X)
        #Error
        cost = loss(hypothesis,Y)
        # Backparopagation
        cost.backward()
        #cal weight
        optimzier.step()
        #avr cal
        avg_cost = avg_cost + cost/batch_len
      #if epoch % (nb_epoch / 11) == 0:
      #print("%f"%cost.item())
    ## ACC
  print("LR :  %.0E Cost : %f"%(LR,cost.item()))

  0%|          | 0/11 [00:55<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
data_loader_test = torch.utils.data.DataLoader(dataset=mnist_test) #batch size at parameters)
with torch.no_grad():
  model.eval()
  wrong_cnt = 0
  for X_test,Y_test in data_loader_test:
    X_test = X_test.to(device)
    Y_test = Y_test.to(device)
    if torch.argmax(model(X_test)) != Y_test:
      wrong_cnt += 1
  print((len(data_loader_test)-wrong_cnt)/len(data_loader_test)*100,"%% accuracy")

98.42 %% accuracy


It didnt took me a lot to code. Because only the model changed  
Learning how to do it took a long time though.

--update 2023/01/29 1628i--  
used AlexTransform for getter result  

2023/01/29 0309i  
I have to go to sleep